In [2]:
import numpy as np
#from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from skimage.feature import hog
import imutils
import argparse
import cv2

In [4]:
# Lee los datos y guárdalos en data
data = np.load("train_signs_detection.npy")
Xtrain = data[:, :-1]
ytrain = data[:, -1].reshape(-1, 1)

#rn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
#rn.fit(Xtrain, ytrain)

NB = GaussianNB()
NB.fit(Xtrain, ytrain)

C:\Users\porta\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()

In [5]:
def pyramid(image, scale=1.5, min_size=(30, 30)):
    """
    Genera imágenes sucesivamente más pequeñas con base a los parámetros proporcionados.
    :param image: Imagen base.
    :param scale: Factor de reducción de cada nivel de la pirámide.
    :param min_size: Dimensiones mínimas que debe tener un nivel de la pirámide.
    :return: Un nuevo nivel de la pirámide en cada llamada.
    """
    yield image  # La primera capa de la pirámide será la imagen original.

    while True:
        # Reducimos el ancho de la imagen en base a 'scale', y luego la
        # redimensionamos manteniendo la relación de aspecto.
        width = int(image.shape[1] / scale)
        image = imutils.resize(image, width=width)

        # Si el resultado es una imagen más pequeña de lo esperada, rompemos el bucle y salimos.
        if image.shape[0] < min_size[1] or image.shape[1] < min_size[0]:
            break

        # Devuelve el siguiente nivel de la pirámide.
        yield image


def sliding_window(image, step_size, window_size):
    """
    Genera ventanas de tamaño `window_size` hasta haber recorrido toda la imagen.
    :param image: Imagen sobre la cual generar ventanas.
    :param step_size: Número de píxeles que nos moveremos en cada dirección.
    :param window_size: Dimensiones de la ventana deslizante.
    :return: Coordenadas (x, y) de la esquina superior izquierda de la ventana, y la región como tal.
    """
    for x in range(0, image.shape[0], step_size):
        for y in range(0, image.shape[1], step_size):
            yield x, y, image[x:x + window_size[1], y:y + window_size[0]]

In [19]:
# Cargamos la imagen en memoria.
rutaImg = 'Dataset_images/00002.jpg'
image = cv2.imread(rutaImg, 0)
imagenColor = cv2.imread(rutaImg)

#Array de ventanas que han detectado una señal para luego mostrarlas
detections = []
tam = 100
#Iteramos cada nivel/imagen de la pirámide
for index, layer in enumerate(pyramid(image, scale=1.5)):
    print('Pyramid layer {}'.format(index))
    # ... y en cada uno deslizaremos una ventana con las proporciones especificadas, moviéndonos 32 píxeles a la vez.
    for x, y, window in sliding_window(layer, step_size=32, window_size=(tam, tam)):
        
        #Cumple si la ventana es de 100x100 y si la intensidad media de ésta es menor que 230.
        if window.shape == (tam,tam) and np.mean(window) < 230:
            acc = NB.predict_proba(hog(cv2.equalizeHist(window)).reshape(1,-1))
            
            #Solo consideramos las ventanas con una probabilidad de positivo superior al 0.9
            if acc[0,1] > 0.9:       
                clone = layer.copy()
                
                #Guardamos los datos necesarios de la ventana para mostrarla al final.
                detections.append((x/layer.shape[0], y/layer.shape[1], 100/layer.shape[0], acc[0,1]))

                print('Coordenadas ventana imagen grande: {} {}'.format(x,y))
                #Dibujamos la ventana.
                cv2.rectangle(clone, (y,x), (y + 100, x + 100), (0, 128, 0), 2)
                
                #Creamos una etiqueta con la probabilidad.
                label = "{}: {:.2f}%".format('Sign', acc[0,1] * 100)
                label2 = 'Coordenadas -> x: ' + str(y) + ' y: ' + str(x)
                y2 = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(clone, label, (y2, x), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
                cv2.putText(clone, label2, (150, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
            
#Una vez procesado todo...
#Convertimos la lista en un array para mayor facilidad a la hora de operar.
detections = np.array(detections)     
clone = imagenColor.copy()

#detections = nms(detections, imagenColor.shape, 0.3)

for i in range(0,detections.shape[0]):
    
    #Consigo las coordenadas donde comenzaría la ventana en la imagen original.
    x = int(detections[i,0]*clone.shape[0]) 
    y = int(detections[i,1]*clone.shape[1])
    
    #Obtenemos ancho y alto de la ventana dimensionados a la imagen original.
    tamVentana = int(detections[i,2]*clone.shape[0])
    
    #Sacamos la probabilidad de la predicción de esa ventana.
    precision = detections[i,3]
    
    #Dibujamos la ventana.
    cv2.rectangle(clone, (y,x), (y + tamVentana, x + tamVentana), (0, 128, 0), 2)
    label = "{}: {:.2f}%".format('Sign', precision * 100)
    y2 = y - 20 if y - 20 > 15 else y + 15
    cv2.putText(clone, label, (y2, x), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,128,0), 2)
    
#Mostramos la imagen.
cv2.imshow('Ventana', clone) 
cv2.waitKey(0)
cv2.destroyAllWindows()

Pyramid layer 0
Coordenadas ventana imagen grande: 448 960
Coordenadas ventana imagen grande: 448 992
Coordenadas ventana imagen grande: 448 1024
Coordenadas ventana imagen grande: 480 1024
Pyramid layer 1
Coordenadas ventana imagen grande: 288 640
Pyramid layer 2
Coordenadas ventana imagen grande: 192 416
Pyramid layer 3
Pyramid layer 4
Pyramid layer 5
Pyramid layer 6
Pyramid layer 7


TypeError: only size-1 arrays can be converted to Python scalars

In [18]:
def nms(detections, dimensionesOriginal, umbral):
    if len(detections) == 0:
        return []
        
    #Inicializo lista de las ventanas que van a "sobrevivir"
    elegidos = []
    
    #Obtenemos anchos y altos de las ventanas dimensionados a la imagen original.
    tamVentana = int(detections[:,2].reshape(1,-1)*dimensionesOriginal[0])
    
    #Consigo las coordenadas de las ventanas en la imagen original.
    x1 = int(detections[:,0]*dimensionesOriginal[0]) 
    y1 = int(detections[:,1]*dimensionesOriginal[1])
    x2 = x1 + tamVentana
    y2 = y1 + tamVentana
    
    #Sacamos las probabilidades de la predicción de esa ventana.
    precision = detections[:,3]
    
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        
        last = len(idxs) - 1
        i = idxs[last]
        elegidos.append(i)
        suppress = [last]
    
        for pos in xrange(0, last):
            
            j = idxs[pos]
            
            xx1 = max(x1[i], x1[j])
            yy1 = max(y1[i], y1[j])
            xx2 = min(x2[i], x2[j])
            yy2 = min(y2[i], y2[j])
            
            w = max(0, xx2 - xx1 + 1)
            h = max(0, yy2 - yy1 + 1)
            
            overlap = float(w * h) / area[j]
            
            if overlap > umbral:
                suppress.append(pos)
        
        idxs = np.delete(idxs, suppress)
        
    return predictions[elegidos]